## 4-circle diffractometer

Shamelessly cribbed from Pete Jemian's bluesky lesson 7

https://github.com/BCDA-APS/use_bluesky/blob/master/lessons/lesson7.ipynb.

#### Preparation
Import the `gi` package before importing `hkl`. The `gi` package is installed automatically if `hklpy`
is installed with `conda` from the `nsls2forge` channel. For example:
```
conda create -n hkl python=3.7
conda activate hkl
conda install hklpy -c nsls2forge
python
>>> import gi
>>> +
```

In [ ]:
import gi
gi.require_version("Hkl", "5.0")

In [ ]:
from hkl.diffract import E4CV
from hkl.util import Lattice

import bluesky.plans as bp
import bluesky.plan_stubs as bps

from ophyd import Component, Device, PseudoSingle, SoftPositioner

In [ ]:
class FourCircleDiffractometer(E4CV):
    h = Component(PseudoSingle, '',
        labels=("hkl", "fourc"), kind="hinted")
    k = Component(PseudoSingle, '',
        labels=("hkl", "fourc"), kind="hinted")
    l = Component(PseudoSingle, '',
        labels=("hkl", "fourc"), kind="hinted")

    omega = Component(SoftPositioner,
        labels=("motor", "fourc"), kind="hinted")
    chi =   Component(SoftPositioner,
        labels=("motor", "fourc"), kind="hinted")
    phi =   Component(SoftPositioner,
        labels=("motor", "fourc"), kind="hinted")
    tth =   Component(SoftPositioner,
        labels=("motor", "fourc"), kind="hinted")

    def __init__(self, *args, **kwargs):
        """
        start the SoftPositioner objects with initial values

        Since this diffractometer uses simulated motors,
        prime the SoftPositioners (motors) with initial values.
        Otherwise, with position == None, describe() and
        other functions get confused.
        """
        super().__init__(*args, **kwargs)

        for axis_motor in self.real_positioners:
            axis_motor.move(0)

In [ ]:
fourc = FourCircleDiffractometer(prefix="", name="fourc")

In [ ]:
print(fourc.omega.position)

In [ ]:
fourc.omega.move(1)

In [ ]:
fourc.omega.position

#### Operating Mode

In [ ]:
fourc.calc.engine.mode

In [ ]:
fourc.engine.modes


In [ ]:
fourc.calc.engine.mode = "constant_phi"
print(fourc.calc.engine.mode)

In [ ]:
fourc.calc.engine.mode = "bissector"
print(fourc.calc.engine.mode)

#### Wavelength

In [ ]:
fourc.calc.wavelength


In [ ]:
fourc.calc.wavelength = 1.62751693358


#### Sample

In [ ]:
fourc.calc.sample

In [ ]:
fourc.calc.new_sample('orthorhombic',
    lattice=Lattice(
        a=1, b=2, c=3,
        alpha=90.0, beta=90.0, gamma=90.0))

In [ ]:
fourc.calc._samples

In [ ]:
fourc.calc.sample_name

In [ ]:
fourc.calc.sample = "main"
fourc.calc.sample.name

In [ ]:
fourc.calc.new_sample(
    "EuPtIn4_eh1_ver",
    lattice=Lattice(
        a=4.542,
        b=16.995,
        c=7.389,
        alpha=90.0,
        beta=90.0,
        gamma=90.0
    )
)

In [ ]:
fourc.calc.sample.U

In [ ]:
fourc.calc.sample.UB

Set the U matrix

In [ ]:
fourc.calc.sample.U = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
]
print(f"[U]:\n{fourc.calc.sample.U}")
print(f"[UB]:\n{fourc.calc.sample.UB}")

The UB matrix can also be assigned a new value.

#### Reflections
A reflection associates a set of reciprocal-space axes (hkl) with a set of real-space motor positions. Following the method of Busing & Levy (Acta Cryst (1967) 22, pp 457-464), two reflections are used to calculate an orientation matix (UB matrix) which is used to convert between motor positions and hkl values.

There are no reflections defined by default.

Define a reflection by associating a known hkl reflection with a set of motor positions.

In [ ]:
rp1 = fourc.calc.Position(omega=22.31594, chi=89.1377, phi=0, tth=45.15857)
r1 = fourc.calc.sample.add_reflection(0, 8, 0, position=rp1)

Define a second reflection (that is not a multiple of the first reflection):

In [ ]:
rp2 = fourc.calc.Position(omega=34.96232, chi=78.3139, phi=0, tth=71.8007)
r2 = fourc.calc.sample.add_reflection(0, 12, 1, position=rp2)

Calculate the UB matrix from these two reflections.

In [ ]:
fourc.calc.sample.compute_UB(r1, r2)
print(fourc.calc.sample.UB)